In [9]:
import numpy as np
import sys

In [10]:
def gaussian_psf(x, y, sigma, xcen, ycen, samp):
    amp = samp
    d = np.sqrt(np.power(np.subtract(x, xcen), 2) +
                np.power(np.subtract(y, ycen), 2))
    g = np.exp(-(np.divide(np.power(d, 2.0), (2.0 * np.power(sigma, 2.0)))))
    g = np.divide(g, np.sum(g))
    g = np.multiply(amp, g)
    return g

In [11]:
# take the star's and sky's photon flux and brightness respectively and return the star's intensity and the sky's flux
def bright(insky, instar, dtel, field, resol, texp):
    # insky: sky brightness in ph/s/cm^2/arcsec^2
    # instar: star flux in ph/s/cm^2
    # dtel: telescope diameter in cm
    # field: field of view in deg
    pixn = np.power(resol, 2.0)               # total pixels
    fov = field*60.0*60.0                     # field of view in arcsec
    telarea = np.pi*0.25*np.power(dtel, 2.0)  # telescope effective area
    # apparent sky flux in ph/px^2
    fB = insky*telarea*np.power(fov, 2.0)*np.power(pixn, -1.0)*texp
    # apparent star intensity in ph
    fS = instar*telarea*texp
    return fB, fS

In [12]:
# create the star positions
def mkfields(fields, field):
    # if the fields variable is a string saying 'set' then the fields produced are a homogeneous sampling of the image
    if fields == 'set':
        posx = [(0.7*field), (0.3*field), (0.3*field), (0.7*field), (0.1*field), (0.1*field),
                (0.1*field), (0.5*field), (0.5*field), (0.5*field), (0.9*field), (0.9*field), (0.9*field)]
        posy = [(0.7*field), (0.7*field), (0.3*field), (0.3*field), (0.1*field), (0.5*field),
                (0.9*field), (0.1*field), (0.5*field), (0.9*field), (0.1*field), (0.5*field), (0.9*field)]
    # else, the fields are at random x,y positions
    elif isinstance(fields, int):
        fields = int(fields)
        posx = []
        posy = []
        for i in range(0, fields):
            maker = np.random.rand(1, 2)
            maker = maker[0]
            curx = maker[0]
            cury = maker[1]
            curx *= field
            cury *= field
            posx.append(curx)
            posy.append(cury)
    else:
        print('Wrong "fields" variable')
        sys.exit(1)
    return posx, posy

In [1]:
# show an image in plotly
def showim(image, fname='test'):
    import plotly.graph_objs as go
    from plotly import __version__
    from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
    init_notebook_mode(connected=True)
    tickval = round(((len(image)+len(image))/2)/100)*10
    data = [go.Heatmap(z=image, x0=0, y0=0, dx=1, dy=1, colorscale='Greys')]
    layout = go.Layout(height=600, width=600, xaxis=dict(autotick=False, ticks='outside', tick0=0, dtick=tickval), yaxis=dict(
        autotick=False, ticks='outside', tick0=0, dtick=tickval, scaleanchor='x'))
    fig = go.Figure(data=data, layout=layout)
    plot(fig, filename=(fname+'.html'))

In [14]:
def mkbias(nllvl, maxc, resol):
    lam = (nllvl/100.0)*maxc
    return np.random.poisson(lam=lam, size=(resol, resol))  # bias frame in ADU

In [15]:
def mkdark(darkl, texpo, resol):
    texpo /= 3600
    lam = darkl*texpo
    # dark frame in electrons
    return np.random.poisson(lam=lam, size=(resol, resol))

In [1]:
def mkrdns(rdns, resol):
    lam = np.power(rdns, 2.0)
    # readnoise frame in electrons
    return np.subtract(np.random.poisson(lam=lam, size=(resol, resol)), lam)